# Neural Networks in Pure Python

In [ ]:
from __future__ import annotations

In [ ]:
class Matrix:
    "Matrix implementation of dimensions 1 or 2"
    def __init__(self, x):
        self._x = x
        self.g = None

    def __getitem__(self, idxs):
        if isinstance(idxs, Matrix):
            rows, cols = self.shape()
            return Matrix([
                [self[i,j] if bool(idxs[i,j]) else 0. for j in range(cols)]
                for i in range(rows)])
        else:
            cur = self._x
            if isinstance(idxs, int):
                idxs = [idxs]
            for ix in idxs:
                cur = cur[ix]
            return cur
        raise NotImplementedError("invalid idxs")

    def __repr__(self):
        return f"Matrix {self.shape()}\n{self._x}"

    def ndim(self):
        dims = 0
        cur = self._x
        while True:
            if isinstance(cur, list):
                cur = cur[0]
                dims+=1
            else:
                return dims
                
    def shape(self):
        s = []
        cur = self._x
        for _ in range(self.ndim()):
            s.append(len(cur))
            cur = cur[0]
        return tuple(s)
        
    def apply(self, f):
        rows, cols = self.shape()
        return Matrix([[f(self[i,j]) for j in range(cols)] for i in range(rows)])

    def reduce(self, f, acc=0):
        for x in self.flatten():
            acc = f(acc, x)
        return acc
        
    def op(self, f, other):
        rows, cols = self.shape()
        if isinstance(other, Matrix):
            assert self.shape() == other.shape(), f"got shapes: {self.shape()} {other.shape()}"
            return Matrix([[f(self[i,j], other[i,j]) for j in range(cols)] for i in range(rows)])
        elif isinstance(float(other), float):
            return Matrix([[f(self[i,j], other) for j in range(cols)] for i in range(rows)])
        
    def transpose(self):
        rows, cols = self.shape()
        new_x = Matrix.zeros(cols, rows)
        for i in range(rows):
            for j in range(cols):
                new_x._x[j][i] = self[i, j]
        return new_x

    def __matmul__(self, other):
        # (x, d) @ (d, y)
        (x, d_x), (d_y, y) = self.shape(), other.shape()
        assert d_x == d_y, f"got shapes: {self.shape()} {other.shape()}"; d = d_x
        res = Matrix([[0 for _ in range(y)] for _ in range(x)])
        # y_t = other.transpose() # y, d
        for i in range(x):
            for j in range(y):
                
                # dot(x[i], other[:, j])
                # transposed matrix swapped rows and cols
                # res._x[i][j] = dot(self._x[i], y_t[j])
                
                # or alternatively:
                for k in range(d):
                    res._x[i][j] += self[i, k] * other[k, j]
        return res


    def expand_dims(self, other):
        cur_rows, cur_cols = self.shape()
        rows, cols = other.shape()

        res = Matrix.zeros_like(other)

        if cur_cols == 1 and cols != 1:
            # expand cols
            
            
        if cur_rows == 1 and rows != 1:
            # expand rows

    
    def dot(self, y):
        assert self.ndim() == 1 and y.ndim() == 1
        return sum(i*j for i, j in zip(self._x, y._x))

    def __add__(self, y):
        return self.op(lambda x,y: x + y, y)
        
    def __mul__(self, y):
        return self.op(lambda x,y: x * y, y)
        
    def __sub__(self, y):
        return self.op(lambda x,y: x - y, y)
        
    def __div__(self, y):
        return self.op(lambda x,y: x / y, y)

    def flatten(self):
        return Matrix([x for row in self._x for x in row])
        
    def zero_grad_():
        self.g = None
        
    @staticmethod
    def zeros(rows, cols):
        return Matrix([[0 for _ in range(cols)] for _ in range(rows)])

    @staticmethod
    def zeros_like(mat): return Matrix.zeros(*mat.shape())
        

IndentationError: expected an indented block after 'if' statement on line 93 (2921377912.py, line 97)

In [ ]:
a = Matrix([[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11]])
a * a

In [ ]:
mask = a.apply(lambda x: x>6); mask

In [ ]:
a.ndim()

In [ ]:
a.flatten()

In [ ]:
a[mask]

In [ ]:
a @ a.transpose()

In [ ]:
a * 2

In [ ]:
a.flatten()

In [ ]:
(a+1).reduce(lambda acc,x: acc * x, 1)

In [ ]:

class Module:
    def __init__(self): 
        self.modules = {}
        self.g = 0

    def __setattr__(self, k, v):
        if not k.startswith("_"):
            self.modules[k] = v
        super().__setattr__(k,v)

    def named_children(self):
        yield from self.modules.items()

    def parameters(self):
        for l in self.modules.values():
            yield from l.parameters()
    
    def forward(self, *args): raise NotImplementedError()
    def zero_grad_(self): self.g = 0
        
    def __call__(self, *inp: list[Matrix]):
        self.inp = inp
        self.out = self.forward(*inp)
        return self.out

    def backward():
        self.bwd(self.out, *self.inp)
        # for i in self.inp:
            # i.backward()
            
        
class Relu(Module):
    def forward(self, x: Matrix): return x.apply(lambda i: max(0., i))
    def bwd(self, out, inp): inp.g += (inp>0) * out.g

class Linear(Module):
    def __init__(self, w: Matrix, b: Matrix):
        super().__init__()
        self.w = w
        self.b = b
    
    def forward(self, x: Matrix): return x @ self.w.transpose() + self.b
        
    def bwd(self, out, inp):
        inp.g += out.g @ self.w.transpose()
        self.w.g += out.g @ x.transpose()
        self.b.g += out.g

class MSE(Module):
    def forward(self, inp: Matrix, targ: Matrix):
        self.diff = (inp - targ)
        xsq = self.diff.apply(lambda a: a**2)
        return xsq / xsq_sum
    
    def bwd(self, out, inp):
        inp.g += 2 * self.diff

In [ ]:
l1 = Linear(Matrix([[10],[20],[30]]), Matrix([[0,0,0]]))
l1(Matrix([[1],[2],[3]]))

In [ ]:
Matrix([[10,20,30]]).transpose()

In [ ]:
Matrix([[10],[20],[30]]) @ 

In [ ]:
1 + 2